# Geospatial analysis

- Comment: Cartoframes.viz was used during the search in Foursquare API, however, to plot the final results, Foium maps were used

In [1]:
import sys
import os
sys.path.append("../")
from src.set_functions import *
from src.createmaps import *
from dotenv import load_dotenv
load_dotenv()

True

- The tokens for Geocode and foursquare are in the .env file

In [2]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")
geocode_token = os.getenv("geocode_token")

- The following code-box is used to obtain the coordinates of the chosen city, in my case, Birmingham, however, as sometimes Geocode API has problems, the coordinates are given below

In [3]:
#This function returns the city coordinates, if the Token and the website are working it can be used.
where = "birmingham"
geo_city = geocode(where, geocode_token)
lon = geo_city.get("longitude")
lat = geo_city.get("latitude")
city = {'type': 'Point', 'coordinates': [lon, lat]}

- The fist list we want to find is the Starbucks in the city centre, we limit the search to 20

In [4]:
#In case Geocode doesn't work, here are the Birmingham (UK) coordinates:
city = {'type': 'Point', 'coordinates': [52.47646,-1.88076]}
looking_for_1 = "starbucks"
limit = 20

- Comment: dataframes were created to use Cartoframes during the fist steps of the search, I'm not using them anymore, but I didn't want to delete them, in case I need to come back later and use them.

In [5]:
data = find_list_foursquare(city, looking_for_1, tok1, tok2, limit)
final_list_1 = get_coordinates_from_list(data)
gdf = create_table(final_list_1)

- Then we look for a list of schools

In [6]:
looking_for_2 = "school"
limit = 20
data_2 = find_list_foursquare(city, looking_for_2, tok1, tok2, limit)
final_list_2 = get_coordinates_from_list(data_2)
gdf_2 = create_table(final_list_2)

- List of pubs

In [7]:
looking_for_3 = "pub"
limit = 30
data_3 = find_list_foursquare(city, looking_for_3, tok1, tok2, limit)
final_list_3 = get_coordinates_from_list(data_3)
gdf_3 = create_table(final_list_3)

- Finally, list of train stations, it is limited to 5, in the city centre there are 3 main train stations, however, the search in Foursquares is giving me the platforms as results. That's why I'm limiting the search to 5

In [8]:
looking_for_4 = "Train Station"
limit = 5
data_4 = find_list_foursquare(city, looking_for_4, tok1, tok2, limit)
final_list_4 = get_coordinates_from_list(data_4)
gdf_4 = create_table(final_list_4)

- I call the function all_combinations, it takes the 4 list obtained above and returns ALL the possible combinations with one place from each list
- In the src folder there is another function, (i.e n_list_combination), to make the same but with n lists instead of 4, is more general and makes the same but without the limitation of using 4 lists. Both functions can be used, however, I rather use the limited one for safety reasons, as the function closer_places needs 4 arguments. 

In [9]:
polygon = all_combinations(final_list_1, final_list_2, final_list_3, final_list_4)

- I call the function get_optimal_combination. Given the list of possible combinations, it calculates the distance between the 4 places (Perimeter of the 4-sided concave polygon and the 2 diagonals), sums the 6 distances and returns the combination with the smallest sum of distances between the places

In [10]:
closer_places = get_optimal_combination(polygon)

- The optimal coordinates are shown as follows:

In [11]:
closer_places

[[52.479158, -1.89932],
 [52.48083, -1.8973938],
 [52.47917395373524, -1.8986854974151004],
 [52.47775090364966, -1.899928107174874]]

- The coordinates given above correspond to the following list of tags, I preffer to call them as they were looked for during the search. It will avoid potential errors and misunderstandings with the names given by the Foursquares API, for example, a place returned from the API could be named "St. Patrick", but we don't know if it's a pub, a train station...

In [12]:
tags = [looking_for_1, looking_for_2, looking_for_3, looking_for_4]
tags

['starbucks', 'school', 'pub', 'Train Station']

- Calling the function get_centroid_place:
    - Given the coordinates of the optimal combination of places, now I calculate their centroid, obtaining the place between them with smaller distance to the 4 positions 
    - The optimal place of the optimal set of combinations, so this is the place chosen, and the office shold be there (Obviously this is the CEO' decision, and if there were a park in the middle or something else we should move it to the side, but the optimal place is there

In [13]:
the_place = get_centroid_place(closer_places)
the_place

[52.47922821434622, -1.8988318511474938]

- The places are shown in the following map

In [14]:
get_the_map(closer_places, the_place, tags)